In [67]:
import os
import numpy as np 
import operator
def get_item_info(item_file):
    '''
    get item title and genres
    return:
    a dict,key itemid,value:[title,genres]
    '''
    if not os.path.exists(item_file):
        return {}
    item_info={}
    f=open(item_file,'r',encoding='ISO-8859-1')
    for line in f:
        item=line.strip().split('::')
        if len(item)<3:
            continue
        [itemid,title,genres]=item
        if itemid not in item_info:
            item_info[itemid]=[title,genres]
    f.close()
    return item_info
def get_ave_score(input_file):
    if not os.path.exists(input_file):
        return {}
    f=open(input_file,'r',encoding='ISO-8859-1')
    temp_dict={}
    score_dict={}    
    for line in f:
        score=line.strip().split('::')
        userid,itemid,rating=score[0],score[1],int(score[2])
        if itemid not in temp_dict:
            temp_dict[itemid]=[0,0]
        temp_dict[itemid][0]+=1
        temp_dict[itemid][1]+=rating
    f.close()
    for itemid in temp_dict:
        score_dict[itemid]=round(temp_dict[itemid][1]/temp_dict[itemid][0],3)
    return score_dict
def get_train_data(file):
    f=open(file,'r',encoding='ISO-8859-1')
    neg_dict={}
    pos_dict={}
    train_data=[]
    score_dict=get_ave_score(file)
    for line in f:
        item=line.strip().split('::')
        userid,itemid,rating=item[0],item[1],float(item[2])
        if userid not in pos_dict:
            pos_dict[userid]=[]
        if userid not in neg_dict:
            neg_dict[userid]=[]
        if rating>=4:
            pos_dict[userid].append((itemid,1))
        else:
            score =score_dict.get(itemid,0)
            neg_dict[userid].append((itemid,score))
    f.close()
    for userid in pos_dict:
        data_num=min(len(pos_dict[userid]),len(neg_dict.get(userid,[])))
        if data_num>0: 
            train_data+=[(userid,zuhe[0],zuhe[1]) for zuhe in pos_dict[userid]][:data_num]
        else:
            continue
        sorted_neg_list=sorted(neg_dict[userid],key=lambda x:x[1],reverse=True)[:data_num]
        train_data+=[(userid,zuhe[0],0) for zuhe in sorted_neg_list ]
    return train_data
def lfm_train(train_data,F,alpha,beta,step):
    user_vector={}
    item_vector={}
    for step_index in range(step):
        for data_instance in train_data:
            userid,itemid,label=data_instance
            if userid not in user_vector:
                user_vector[userid]=init_model(F)
            if itemid not in item_vector:
                item_vector[itemid]=init_model(F)
        delta=label-model_predict(user_vector[userid],item_vector[itemid])
        for f in range(F):
            user_vector[userid][f]+=beta*(delta*item_vector[itemid][f]-alpha*user_vector[userid][f])
            item_vector[itemid][f]+=beta*(delta*user_vector[userid][f]-alpha*item_vector[itemid][f])
        beta*=0.9
    return user_vector,item_vector
            
            
def init_model(F):
    return np.random.randn(F)
def model_predict(user_vector,item_vector):
    res=np.dot(user_vector,item_vector)/(np.linalg.norm(user_vector)*np.linalg.norm(item_vector))
    return res

    
#def RandomselectNegetiveSample(items):
def give_recom_result(user_vector,item_vector,userid):
    if userid not in user_vector:
        return[]
    record={}
    user_vec=user_vector[userid]
    recom_list=[]
    for itemid in item_vector:
        item_vec=item_vector[itemid]
        res=np.dot(user_vec,item_vec)/(np.linalg.norm(user_vec)*np.linalg.norm(item_vec))
        record[itemid]=res
    for i in sorted(record.items(),key=operator.itemgetter(1),reverse=True)[:10]:
        itemid=i[0]
        recom_list.append((itemid,i[1]))
    return recom_list
def model_train():
    path='C:Desktop/CF/data/ratings.dat'
    train_data=get_train_data(path)
    user_vector,item_vector=lfm_train(train_data,50,0.01,0.1,50)
    return give_recom_result(user_vector,item_vector,'1')
#def ana_recom_result(train_data,userid,recom_list):
model_train()


[('3284', 0.4802093857088489),
 ('3532', 0.4471791657505729),
 ('2470', 0.4375469692974414),
 ('3281', 0.4070810212574792),
 ('1445', 0.40332554915499075),
 ('1034', 0.4004305379542357),
 ('3077', 0.38796090475219436),
 ('2621', 0.387170160241237),
 ('2189', 0.3852725599883872),
 ('1983', 0.3791595103461105)]